In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('insurance.csv')

In [8]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [9]:

df.region.replace({'southwest':0, 'southeast':1, 'northwest':2, 'northeast':3})
df.sex.replace({'female':0, 'male':1})
df.smoker.replace({'yes':1, 'no':0})

0       1
1       0
2       0
3       0
4       0
       ..
1333    0
1334    0
1335    0
1336    0
1337    1
Name: smoker, Length: 1338, dtype: int64

In [32]:
import re
from this import d
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

class Tree_algorithms:

    def preprocess():
        ord = OrdinalEncoder()
        data = 'insurance.csv'
        df = pd.read_csv(data)
        df[['sex', 'smoker', 'region']] = ord.fit_transform(df[['sex', 'smoker', 'region']])

        return df
        
    def dec_trees():
        df = Tree_algorithms.preprocess()
        y = df['smoker']
        X = df.drop(['smoker'], axis=1)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

        dctr = DecisionTreeRegressor(random_state=0)
        dctr.fit(X_train, y_train)

        train = dctr.score(X_train, y_train)
        test = dctr.score(X_test, y_test)
        return (f'training accuracy:{train}\n testing accuracy: {test}')
    
    def rand_forest():
        df = Tree_algorithms.preprocess()
        y = df['smoker']
        X = df.drop(['smoker'], axis=1)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

        randf = RandomForestRegressor(random_state=0)
        randf.fit(X_train, y_train)

        train = randf.score(X_train, y_train)
        test = randf.score(X_test, y_test)
        return (f'training accuracy:{train}\n testing accuracy: {test}')
        
    def ext_rand_tress():
        df = Tree_algorithms.preprocess()
        y = df['smoker']
        X = df.drop(['smoker'], axis=1)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

        ex_rand_tr = ExtraTreesRegressor(random_state=0)
        ex_rand_tr.fit(X_train, y_train)

        train = ex_rand_tr.score(X_train, y_train)
        test = ex_rand_tr.score(X_test, y_test)
        return (f'training accuracy:{train}\n testing accuracy: {test}')
        
    def adaboosting():
        df = Tree_algorithms.preprocess()
        y = df['smoker']
        X = df.drop(['smoker'], axis=1)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

        adab = AdaBoostRegressor(random_state=0)
        adab.fit(X_train, y_train)

        train = adab.score(X_train, y_train)
        test = adab.score(X_test, y_test)
        return (f'training accuracy:{train}\n testing accuracy: {test}')
        
    


print(Tree_algorithms.ext_rand_tress())

training accuracy:1.0
 testing accuracy: 0.848029492362983


In [ ]:
df[df['olma']>2]